# 15min RSI Scalping Strategy

In [1]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_15_M_BID_31.01.2019-30.01.2022.csv")
df.head()

,Local time,Open,High,Low,Close,Volume
0,31.01.2019 00:00:00.000 GMT+0200,1.14786,1.14857,1.14782,1.14829,1170.10
1,31.01.2019 00:15:00.000 GMT+0200,1.14829,1.14858,1.14823,1.14849,1842.58
2,31.01.2019 00:30:00.000 GMT+0200,1.14853,1.14854,1.14813,1.14819,1002.61
3,31.01.2019 00:45:00.000 GMT+0200,1.14818,1.14828,1.14804,1.14805,609.26
4,31.01.2019 01:00:00.000 GMT+0200,1.14809,1.14850,1.14809,1.14849,1669.71


In [2]:
df=df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78


In [3]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.Close, length=100)
df["RSI"] = ta.rsi(df.Close, length=3)
a=ta.adx(df.High, df.Low, df.Close, length=7)
df['ADX']=a['ADX_7']
df['ATR']= df.ta.atr()
#help(ta.adx)

# EMA Trend Signal

In [4]:
emasignal = [0]*len(df)
backcandles = 8

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMAsignal'] = emasignal

# RSI & ADX Signal

In [5]:
RSIADXSignal = [0] * len(df)
for row in range(0, len(df)):
    RSIADXSignal[row] = 0
    if df.EMAsignal[row]==1 and df.RSI[row]>=80 and df.ADX[row]>=30:
        RSIADXSignal[row]=1
    if df.EMAsignal[row]==2 and df.RSI[row]<=20 and df.ADX[row]>=30:
        RSIADXSignal[row]=2

df['RSIADXSignal']=RSIADXSignal

# Candles Signal

In [6]:
CandleSignal = [0] * len(df)
for row in range(1, len(df)):
    CandleSignal[row] = 0
    #if (RSIADXSignal[row]==1 or RSIADXSignal[row-1]==1) and (df.Open[row]>df.Close[row]):# and df.Close[row]<df.Close[row-1]):
    #    CandleSignal[row]=1
    #if (RSIADXSignal[row]==2 or RSIADXSignal[row-1]==2) and (df.Open[row]<df.Close[row]):# and df.Close[row]>df.Close[row-1]):
    #    CandleSignal[row]=2
    if RSIADXSignal[row-1]==1 and df.Open[row]>df.Close[row] and df.Close[row]<min(df.Close[row-1], df.Open[row-1]):
        CandleSignal[row]=1
    if RSIADXSignal[row-1]==2 and df.Open[row]<df.Close[row] and df.Close[row]>max(df.Close[row-1], df.Open[row-1]):
        CandleSignal[row]=2

df['TotSignal']=CandleSignal

# Visualizing Signals

In [7]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-4
    elif x['TotSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[0:1000]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='red', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [26]:
df.iloc[667:672]

,Local time,Open,High,Low,Close,Volume,EMA,RSI,EMAsignal,ADX,ATR,RSIADXSignal,TotSignal,pointpos
667,08.02.2019 22:45:00.000 GMT+0200,1.13228,1.13228,1.13211,1.13222,2699.72,1.133655,27.109844,1,38.327831,0.000405,0,0,NaN
668,08.02.2019 23:00:00.000 GMT+0200,1.13222,1.13236,1.13222,1.13232,1140.35,1.133629,55.275499,1,36.121201,0.000386,0,0,NaN
669,08.02.2019 23:15:00.000 GMT+0200,1.13233,1.13238,1.13227,1.13227,1282.86,1.133602,42.855561,1,33.621538,0.000367,0,0,NaN
670,08.02.2019 23:30:00.000 GMT+0200,1.13227,1.13250,1.13226,1.13245,1713.53,1.133579,74.181742,1,29.465928,0.000358,0,0,NaN
671,08.02.2019 23:45:00.000 GMT+0200,1.13244,1.13246,1.13207,1.13207,1169.02,1.133549,27.113760,1,28.566670,0.000360,0,0,NaN


## Stop Loss from Price

In [8]:
SLSignal = [0] * len(df)
SLbackcandles = 2
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.TotSignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.High[i])
        SLSignal[row]=ma
    if df.TotSignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.Low[i])
        SLSignal[row]=mi
        
df['SLSignal']=SLSignal

## Backtesting Strategy

In [9]:
dfpl = df[:]
def SIGNAL():
    return dfpl.TotSignal

In [10]:
from backtesting import Strategy
from backtesting import Backtest
class MyStrat(Strategy):
    initsize = 0.02
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.3*self.data.ATR[-1]
        TPSLRatio = 1.3

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    6.120675
Equity Final [$]                    99.958615
Equity Peak [$]                    101.630183
Return [%]                          -0.041385
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.939875
Avg. Drawdown [%]                    -0.10732
Max. Drawdown Duration                59254.0
Avg. Drawdown Duration            1670.511628
# Trades                                527.0
Win Rate [%]                        43.074004
Best Trade [%]                       0.377823
Worst Trade [%]                      -0.40447
Avg. Trade [%]                    

In [9]:
bt.plot(show_legend=False)

In [12]:
dfpl = df[:]
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.02
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    3.436192
Equity Final [$]                     99.82376
Equity Peak [$]                     101.75591
Return [%]                           -0.17624
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.973183
Avg. Drawdown [%]                    -0.11869
Max. Drawdown Duration                59254.0
Avg. Drawdown Duration            1801.051282
# Trades                                546.0
Win Rate [%]                        50.549451
Best Trade [%]                       0.334823
Worst Trade [%]                     -0.414492
Avg. Trade [%]                    

In [10]:
bt.plot(show_legend=False)